In [1]:
import subprocess
import json
import re
import os

In [2]:
def get_available_cameras():
    """
    Returns a list of available cameras by parsing the output of 'cam -l'.
    Each camera is a dictionary with 'id', 'name', and 'path'.
    """
    try:
        # Run 'cam -l' command
        result = subprocess.run(
            ['cam', '-l'],
            capture_output=True,
            text=True,
            check=True
            )
        output_lines = result.stdout.splitlines()

        cameras = []
        parsing_cameras = False
        for line in output_lines:
            if "Available cameras:" in line:
                parsing_cameras = True
                continue
            if parsing_cameras:
                # Regex to extract camera ID, name, and path
                match = re.match(r'(\d+):\s+\'([^\']+)\'\s+\(([^\)]+)\)', line)
                if match:
                    # cam -l uses 1-based indexing, but rpicam-jpeg uses 0-based
                    cam_id_1_based = int(match.group(1))
                    cam_name = match.group(2)
                    cam_path = match.group(3)
                    
                    # Convert to 0-based index for tools like rpicam-jpeg
                    cam_id_0_based = cam_id_1_based - 1
                    
                    cameras.append({
                        "id_1_based": cam_id_1_based,
                        "id_0_based": cam_id_0_based,
                        "name": cam_name,
                        "path": cam_path
                    })
                elif line.strip() == "": # Stop if an empty line is encountered after camera list
                    break
        return cameras
    except FileNotFoundError:
        print("Error: 'cam' command not found. Ensure libcamera-apps is installed and in your PATH.")
        return []
    except subprocess.CalledProcessError as e:
        print(f"Error running 'cam -l': {e}")
        print(f"Stdout: {e.stdout}")
        print(f"Stderr: {e.stderr}")
        return []


print("--- Available Cameras ---")
cameras = get_available_cameras()
if not cameras:
    print("No cameras found or error occurred.")
else:
    for cam in cameras:
        print(f"  ID (1-based): {cam['id_1_based']}, ID (0-based): {cam['id_0_based']}, Name: '{cam['name']}', Path: '{cam['path']}'")
    print("\n")

    # Example usage:
    # Assuming you want to use the DMM 37UX226-ML camera (which was ID 2 or 0-based ID 1)
    # Verify this ID based on the output of get_available_cameras()
    target_camera_id = None
    for cam in cameras:
        if "arducam_64mp" in cam['name']:
            target_camera_id = cam['id_0_based']
            break
    
    if target_camera_id is None:
        print("arducam_64mp camera not found. Please adjust 'target_camera_id' in the script.")
        # Fallback to first camera if DMM not found, for demonstration
        if cameras:
            target_camera_id = cameras[0]['id_0_based']
            print(f"Defaulting to first camera found: ID {target_camera_id} ({cameras[0]['name']})")
        else:
            exit() # No cameras to work with

--- Available Cameras ---
  ID (1-based): 1, ID (0-based): 0, Name: 'arducam_64mp', Path: '/base/axi/pcie@1000120000/rp1/i2c@80000/arducam_64mp@1a'
  ID (1-based): 2, ID (0-based): 1, Name: 'DMM 37UX226-ML', Path: '/base/axi/pcie@1000120000/rp1/usb@300000-1:1.3-199e:9415'




In [3]:
def capture_image(camera_id_0_based, output_filename, resolution, exposure_us=None, quality=90):
    """
    Captures a single image from the specified camera with given resolution and exposure.
    
    Args:
        camera_id_0_based (int): The 0-based index of the camera to use (e.g., 0 for the first camera).
        output_filename (str): The path to save the JPEG image (e.g., "my_photo.jpg").
        resolution (tuple): A tuple (width, height) for the image resolution.
        exposure_us (int, optional): Manual exposure time in microseconds. If None, auto-exposure is used.
        quality (int): JPEG compression quality (0-100).
    
    Returns:
        bool: True if image capture was successful, False otherwise.
    """
    cmd = [
        'rpicam-jpeg', # Or 'libcamera-still' if you prefer
        '-c', str(camera_id_0_based),
        '--output', output_filename,
        '--width', str(resolution[0]),
        '--height', str(resolution[1]),
        '--quality', str(quality),
        '--timeout', '100' # Give it 100ms to warm up and capture
    ]

    if exposure_us is not None:
        cmd.extend(['--shutter', str(exposure_us)])

    print(f"Executing command: {' '.join(cmd)}")
    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=True
            )
        print(f"Image captured successfully to {output_filename}")
        # print(f"Command stdout:\n{result.stdout}") # Uncomment for debug
        # print(f"Command stderr:\n{result.stderr}") # Uncomment for debug
        return True
    except FileNotFoundError:
        print("Error: 'rpicam-jpeg' command not found. Ensure libcamera-apps is installed and in your PATH.")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Error capturing image: {e}")
        print(f"Stdout: {e.stdout}")
        print(f"Stderr: {e.stderr}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [10]:
print(f"--- Capturing from Camera ID (0-based): {target_camera_id} ---")

# # Capture with auto-exposure and 1920x1080 resolution
# print("\nCapturing auto-exposure image...")
# success = capture_image(target_camera_id, "auto_exposure_1920x1080.png", (1920, 1080))
# if not success:
#     print("Auto-exposure capture failed.")

# # Capture with manual exposure (100,000 microseconds = 0.1 seconds)
# # Note: Your camera's max resolution might differ, adjust as needed.
# # print("\nCapturing manual exposure (0.1s) image at 4000x3000...")
# success = capture_image(target_camera_id, "manual_exposure_4000x3000.png", (4000, 3000), exposure_us=500_000)
# if not success:
#     print("Manual exposure capture failed.")

# Capture with a very short exposure (1000 microseconds = 1ms)
print("\nCapturing short manual exposure (0.001s) image at 1280x720...")
success = capture_image(target_camera_id, "manual_exposure_1280x720_1ms.png", (1280, 720), exposure_us=1_000)
if not success:
    print("Short exposure capture failed.")

--- Capturing from Camera ID (0-based): 0 ---

Capturing short manual exposure (0.001s) image at 1280x720...
Executing command: rpicam-jpeg -c 0 --output manual_exposure_1280x720_1ms.png --width 1280 --height 720 --quality 90 --timeout 100 --shutter 1000
Image captured successfully to manual_exposure_1280x720_1ms.png


In [11]:
def get_available_rpicam_controls_and_formats():
    """
    Parses the --help output of rpicam-still to list common controls,
    and identifies typical output formats.

    Note: This does NOT list all libcamera controls, only those exposed
          as command-line options by rpicam-still.
          Resolutions are generally arbitrary up to sensor max.
    """
    try:
        # Using rpicam-still as it generally has the most comprehensive help
        result = subprocess.run(
            ['rpicam-still', '--help'],
            capture_output=True,
            text=True,
            check=True
            )
        help_output = result.stdout
        print(f'{help_output = }')

        controls = {}
        formats = {
            "still_image_output_formats": [],
            "raw_output_formats": []
        }
        resolutions_info = "Arbitrary resolutions supported up to camera sensor maximum (e.g., 4056x3040 for HQ Camera)."

        # --- Parse Controls ---
        # Look for lines starting with -- or - and containing a description
        # This regex is an attempt to capture option name and its description
        # It's heuristic and might not catch everything perfectly
        
        # Pattern for options like --shutter <value> or --gain <value>
        control_pattern = re.compile(r'^\s*(-{1,2}[\w-]+(?:=\[arg(?:=\[.*\])?\])?(?: <[^>]+>)?)?\s+([^\n]+)', re.MULTILINE)
        
        # A more specific pattern to capture common control lines
        # This will need to be refined based on actual --help output from your system
        # Example line: --shutter <microseconds> Set the shutter speed.
        # Example line: --gain <value> Set the sensor gain.
        # Example line: --awbgains <red>,<blue> Set the AWB gains.
        
        # Let's target the "Options:" section or similar
        options_section_match = re.search(r'(Options:|Common command line options:)\s*(\n.+?)(?=\n[A-Z\s]+:|\n\n|\Z)', help_output, re.DOTALL | re.IGNORECASE)
        
        if options_section_match:
            options_block = options_section_match.group(2)
            
            # This regex is simplified for demonstration and may need tuning.
            # It tries to capture a short option, long option, and description.
            # It's hard to parse arbitrary CLI help text perfectly.
            option_line_pattern = re.compile(r'^\s*(-[a-zA-Z],)?\s*(--[\w-]+(?:\[=arg(?:\[=.*\])?\])?(?: <[^>]+>)?)\s+(.*)', re.MULTILINE)
            
            for line in options_block.splitlines():
                match = option_line_pattern.match(line)
                if match:
                    short_opt = match.group(1) if match.group(1) else ''
                    long_opt = match.group(2).strip()
                    description = match.group(3).strip()
                    
                    # Clean up long_opt to remove optional arg placeholders
                    clean_long_opt = re.sub(r'\[=arg(?:\[=.*\])?\]', '', long_opt).strip()
                    clean_long_opt = re.sub(r'<[^>]+>', '', clean_long_opt).strip()
                    
                    controls[clean_long_opt] = description

        # --- Identify Formats ---
        if "output.jpg" in help_output:
            formats["still_image_output_formats"].append("JPEG (.jpg)")
        if "--raw" in help_output and (".dng" in help_output or "raw Bayer" in help_output):
            formats["raw_output_formats"].append("DNG (raw Bayer) (.dng)")
        # Check for other rpicam tools for more formats
        if "rpicam-vid" in subprocess.check_output(['ls', '/usr/bin/']).decode():
            formats["video_output_formats"] = ["H.264 (.h264)"]
        if "rpicam-raw" in subprocess.check_output(['ls', '/usr/bin/']).decode():
            formats["raw_stream_formats"] = ["Bayer Raw Stream"]

        return {
            "available_controls": controls,
            "available_resolutions_info": resolutions_info,
            "available_output_formats": formats
        }

    except FileNotFoundError:
        print("Error: 'rpicam-still' command not found. Ensure libcamera-apps is installed.")
        return None
    except subprocess.CalledProcessError as e:
        print(f"Error running 'rpicam-still --help': {e}")
        print(f"Stdout: {e.stdout}")
        print(f"Stderr: {e.stderr}")
        return None

if __name__ == "__main__":
    print("--- Camera Capabilities (via rpicam-still --help) ---")
    capabilities = get_available_rpicam_controls_and_formats()

    if capabilities:
        print("\nAvailable Controls:")
        if capabilities["available_controls"]:
            for control_name, description in capabilities["available_controls"].items():
                print(f"  {control_name}: {description}")
        else:
            print("  Could not parse specific controls from help output.")

        print("\nAvailable Resolutions:")
        print(f"  {capabilities['available_resolutions_info']}")

        print("\nAvailable Output Formats:")
        for format_type, format_list in capabilities["available_output_formats"].items():
            if format_list:
                print(f"  {format_type.replace('_', ' ').title()}: {', '.join(format_list)}")
            else:
                print(f"  No {format_type.replace('_', ' ')} found or parsable.")
    else:
        print("Could not retrieve camera capabilities.")

--- Camera Capabilities (via rpicam-still --help) ---
help_output = 'Valid options are:\n  -h [ --help ] [=arg(=1)] (=0)         Print this help message\n  --version [=arg(=1)] (=0)             Displays the build version number\n  --list-cameras [=arg(=1)] (=0)        Lists the available cameras attached to the system.\n  --camera arg (=0)                     Chooses the camera to use. To list the available indexes, use the \n                                        --list-cameras option.\n  -v [ --verbose ] [=arg(=2)] (=1)      Set verbosity level. Level 0 is no output, 1 is default, 2 is verbose.\n  -c [ --config ] [=arg(=config.txt)]   Read the options from a file. If no filename is specified, default to \n                                        config.txt. In case of duplicate options, the ones provided on the command line\n                                        will be used. Note that the config file must only contain the long form \n                                        options